In [1]:
import pandas as pd
import nfl_data_py as nfl

In [2]:
df2 = pd.read_excel('./Data/sportsrefscrape.xlsx')
df2

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,College/Univ,Player_NFL_Link,...,pass_cmp,pass_att,pass_cmp_pct,pass_yds,pass_yds_per_att,adj_pass_yds_per_att,pass_td,pass_int,pass_rating,g
0,2013,1,1,KAN,Eric Fisher,T,22.0,2021.0,Central Michigan,/players/F/FishEr00.htm,...,None,None,None,None,None,None,None,None,None,None
1,2013,1,2,JAX,Luke Joeckel,T,21.0,2017.0,Texas A&M,/players/J/JoecLu00.htm,...,None,None,None,None,None,None,None,None,None,None
2,2013,1,3,MIA,Dion Jordan,DE,23.0,2020.0,Oregon,/players/J/JordDi00.htm,...,None,None,None,None,None,None,None,None,None,NaN
3,2013,1,4,PHI,Lane Johnson,T,23.0,2022.0,Oklahoma,/players/J/JohnLa01.htm,...,None,None,None,None,None,None,None,None,None,None
4,2013,1,5,DET,Ezekiel Ansah,DE,24.0,2020.0,BYU,/players/A/AnsaEz00.htm,...,None,None,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,2022,7,258,GNB,Samori Toure,WR,24.0,2022.0,Nebraska,/players/T/TourSa00.htm,...,None,None,None,None,None,None,None,None,None,None
2554,2022,7,259,KAN,Nazeeh Johnson,SAF,24.0,2022.0,Marshall,/players/J/JohnNa01.htm,...,None,None,None,None,None,None,None,None,None,NaN
2555,2022,7,260,LAC,Zander Horvath,RB,23.0,2022.0,Purdue,/players/H/HorvZa00.htm,...,None,None,None,None,None,None,None,None,None,None
2556,2022,7,261,LAR,AJ Arcuri,OT,25.0,2022.0,Michigan St.,/players/A/ArcuAJ00.htm,...,None,None,None,None,None,None,None,None,None,None


In [3]:
player = nfl.import_players()
draft_players = player.loc[(player.entry_year > 2012) & (player.draft_number >= 1)].sort_values(by=['entry_year','draft_number'])
draft_player = draft_players[['display_name','position_group','height','weight','entry_year','college_conference','draft_number']]

In [ ]:
draft_player

In [4]:
cols = []
cols_remove = ['Tm','Player','Pos','College/Univ','Player_NFL_Link','Player_NCAA_Link','display_name','position_group','college_conference']
num_cols = []
for col in df2:
    cols.append(col)
for i in cols:
    if i not in cols_remove:
        num_cols.append(i)

In [5]:
df2[num_cols] = df2[num_cols].apply(pd.to_numeric,errors='coerce', axis=1)

In [ ]:
df2

In [6]:
df = df2.merge(draft_player,how='left',left_on=['Draft_Yr','Pick','Player'],right_on=['entry_year','draft_number','display_name'])

In [7]:
df[df.duplicated(subset=['Player','Draft_Yr'])]

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,College/Univ,Player_NFL_Link,...,pass_int,pass_rating,g,display_name,position_group,height,weight,entry_year,college_conference,draft_number
1715,2019.0,6.0,188.0,TEN,David Long,LB,22.0,2022.0,West Virginia,/players/L/LongDa04.htm,...,NaN,NaN,NaN,David Long,LB,5-11,224,2019.0,Big Twelve Conference,188.0
2190,2021.0,5.0,154.0,NYJ,Michael Carter,S,22.0,2022.0,Duke,/players/C/CartMi02.htm,...,NaN,NaN,NaN,Michael Carter,DB,5-10,190,2021.0,Atlantic Coast Conference,154.0


In [8]:
df.drop(['g'],axis=1,inplace=True)

In [9]:
df.Rnd[df.Rnd > 1] = 0

C:\Users\capta\AppData\Local\Temp\ipykernel_8908\2056627649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Rnd[df.Rnd > 1] = 0


In [10]:
df.Rnd.value_counts(normalize=True)

0.0    0.875293
1.0    0.124707
Name: Rnd, dtype: float64

In [11]:
df.Pos[df.Pos == 'NT'] = 'DT'

C:\Users\capta\AppData\Local\Temp\ipykernel_8908\698980894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Pos[df.Pos == 'NT'] = 'DT'


In [12]:
df.Pos[(df.Pos == 'T')|(df.Pos == 'OT')|(df.Pos == 'C')|(df.Pos == 'G')] = 'OL'

C:\Users\capta\AppData\Local\Temp\ipykernel_8908\1818363077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Pos[(df.Pos == 'T')|(df.Pos == 'OT')|(df.Pos == 'C')|(df.Pos == 'G')] = 'OL'


In [13]:
df.Pos[df.Pos == 'SAF'] = 'S'

C:\Users\capta\AppData\Local\Temp\ipykernel_8908\465466545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Pos[df.Pos == 'SAF'] = 'S'


In [14]:
df.Pos[(df.Pos == 'OLB')|(df.Pos == 'ILB')] = 'LB'

C:\Users\capta\AppData\Local\Temp\ipykernel_8908\3918856926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Pos[(df.Pos == 'OLB')|(df.Pos == 'ILB')] = 'LB'


In [ ]:
df.Pos.value_counts()

In [ ]:
df[df.Pos == 'DB']

In [15]:
df = df.rename({'College/Univ':'College'},axis=1)

In [17]:
df.weight.astype('float64')

0       315.0
1       307.0
2       275.0
3       317.0
4       275.0
        ...  
2553    190.0
2554    189.0
2555    230.0
2556      NaN
2557    212.0
Name: weight, Length: 2558, dtype: float64

In [18]:
df.to_parquet('./Data/draft1322')